In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\adils\\Documents\\IITP\\summer25\\End-to-End-Medical-Chatbot\\research'

In [3]:
import os
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\adils\\Documents\\IITP\\summer25\\End-to-End-Medical-Chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# Extract Data from the pdf
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [7]:
extracted_data = load_pdf_file("Data/")

In [8]:
#extracted_data

In [9]:
#split the data into text chunks
def split_text(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = split_text(extracted_data)
print("Lenght of text_chunks ", len(text_chunks))

Lenght of text_chunks  5859


In [11]:
#text_chunks

In [12]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [13]:
#Download embeddings from huggingface
def download_hugging_face_embeddings():
    embeddings = HuggingFaceBgeEmbeddings( model_name = "sentence-transformers/all-MiniLM-L6-v2" )
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\adils\AppData\Local\Temp\ipykernel_13288\3939963824.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings( model_name = "sentence-transformers/all-MiniLM-L6-v2" )
c:\Users\adils\miniconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
query_result = embeddings.embed_query("Hello World!")
print("Length" , len(query_result))

Length 384


In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [33]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
COHERE_API_KEY = os.environ.get('COHERE_API_KEY')

In [18]:
from pinecone import Pinecone, ServerlessSpec

import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name = index_name,
    dimension = 384,
    metric = 'cosine',
    spec = ServerlessSpec(
        cloud = 'aws',
        region = "us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'x-cloud-trace-context': '0c0481638e41a92e2753ffd5d8bd04e0', 'date': 'Sun, 01 Jun 2025 07:56:26 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [36]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
#os.environ["COHERE_API_KEY"] =COHERE_API_KEY

In [22]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding = embeddings,
)

In [23]:
# Load from existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embeddings
)

In [24]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs = {"k" :3})

In [25]:
retrieved_docs = retriever.invoke(" What is Acne? ")

In [26]:
retrieved_docs

[Document(id='319bbdef-bc21-42fd-88c8-591ef624a765', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='f2cdab8b-5c88-4620-baea-08d120f82158', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='d1171ea3-0d69-40a4-8d92-9387a292d8ea', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 

In [28]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for  question-answering tasks"
    "Use the following pieces of retrieved context to answer"
    "the question. if you don't know the answer, say that you"
    "don't know.Use three sentences maximum and keep the answer"
    "concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , system_prompt),
        ("human", "{input}"),
    ]
)

In [29]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever , question_answer_chain)

In [37]:
import os
os.environ["COHERE_API_KEY"] = "IDtW285O1k6vzHO0Z7O2C8Awc1b1ftosFvhh3225"

In [57]:
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Use the context below to answer the question. 
If the answer is not in the context, say "I don't know".

Context: {context}
Question: {question}
Answer:"""
)


In [60]:
from langchain.chains import RetrievalQA
from langchain.llms import Cohere

llm = Cohere(model="command", temperature=0.5)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,  # Your vector DB retriever
    return_source_documents=True,
    chain_type_kwargs={"prompt": custom_prompt}
)

query = "what are the symptoms of astigmatism?"
response = rag_chain.invoke({"query": query})



print(response["result"])  # The answer


 The main symptom of astigmatism is blurring. People can also experience headaches and eyestrain. Parents can notice that a child may have astigmatism when the child can see some part of a pattern or picture more clearly than others. For example, lines going across may seem clearer than lines going up and down. 
